In [1]:
import pandas as pd
import numpy as np
from delai.ml_logic.preprocessing import preprocess_X, preprocess_y
from delai.data.local_disk import get_pandas_chunk

In [2]:
df_X = pd.read_csv('../raw_data/initial_X_train.csv')
df_y = pd.read_csv('../raw_data/initial_y_train.csv')

X = preprocess_X(df_X)
y = preprocess_y(df_y)

✅ preprocess_X() done
✅ preprocess_y() done


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(loss='log_loss', average=None)

cv_results = cross_validate(sgd_model, X, y, cv=5, scoring = 'recall')
cv_results


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sameriliya/.pyenv/versions/3.8.12/envs/delai/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sameriliya/.pyenv/versions/3.8.12/envs/delai/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 890, in fit
    return self._fit(
  File "/Users/sameriliya/.pyenv/versions/3.8.12/envs/delai/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 677, in _fit
    if self.average > 0:
TypeError: '>' not supported between instances of 'str' and 'int'
